In [21]:
import nfl_data_py as nfl
import pandas as pd
import matplotlib.pyplot as plt
from adjustText import adjust_text
from matplotlib.patches import Rectangle
import numpy as np

from functions import win_loss, get_team_schedule, fantasy_defense_rankings

In [2]:

week_by_week2024 = nfl.import_weekly_data([2024], downcast=True)
play_by_play2024 = nfl.import_pbp_data([2024], downcast=True, cache=False, alt_path=None)
schedule2024 = nfl.import_schedules([2024])
seasonal2024 = nfl.import_seasonal_data([2024], "REG")
winstotal2024 = nfl.import_win_totals([2024])

Downcasting floats.
2024 done.
Downcasting floats.


In [ ]:
for columns in seasonal2024.columns:
    print(columns)

In [ ]:
# printing the columns of play_byPlay so I don't mess up names lol
for column in play_by_play2024.columns:
    print(column)

In [ ]:
for column in schedule2024.columns:
    print(column)
    

In [ ]:
# printing the columns of week_by_week so I don't mess up names lol
for columns in week_by_week2024.columns: 
    print (columns)


# small_wbw_df = week_by_week2024[["recent_team", "player_name", "fantasy_points_ppr", "wopr", "dakota", "position"]]

# # justin jefferson
# small_wbw_df[(small_wbw_df["recent_team"] == "CIN") & (small_wbw_df["position"] == "WR")]


In [ ]:
def calculate_player_matchup(schedule_df, week_by_week_df, week=0): 
    
    defenseRankings_df = fantasy_defense_rankings(week_by_week_df).set_index("opponent_team")
    
    # creating a dataframe with the team and their matchups for the rest of the season
    matchups_data = []
    listOfTeams = week_by_week_df["recent_team"].unique()
    
    for team in listOfTeams:
        team_matchups = get_team_schedule(schedule_df, team)

        # precompute opponent ranks for each position using dictionary lookups
        qb_opponent_ranks = [defenseRankings_df.at[opponent, "QB_rank"] if opponent in defenseRankings_df.index else np.nan for opponent in team_matchups]
        rb_opponent_ranks = [defenseRankings_df.at[opponent, "RB_rank"] if opponent in defenseRankings_df.index else np.nan for opponent in team_matchups]
        wr_opponent_ranks = [defenseRankings_df.at[opponent, "WR_rank"] if opponent in defenseRankings_df.index else np.nan for opponent in team_matchups]
        te_opponent_ranks = [defenseRankings_df.at[opponent, "TE_rank"] if opponent in defenseRankings_df.index else np.nan for opponent in team_matchups]

        # calculate the average ranks
        qb_opponent_average = np.nanmean(qb_opponent_ranks)
        rb_opponent_average = np.nanmean(rb_opponent_ranks)
        wr_opponent_average = np.nanmean(wr_opponent_ranks)
        te_opponent_average = np.nanmean(te_opponent_ranks)
        
        # append to the matchups list
        matchups_data.append({
            "team": team,
            "matchups": team_matchups,
            "qb_opponent_average": qb_opponent_average,
            "rb_opponent_average": rb_opponent_average,
            "wr_opponent_average": wr_opponent_average,
            "te_opponent_average": te_opponent_average
        })

    matchups_df = pd.DataFrame(matchups_data)

               
    # get multiple dfs of all the positions
    quarterbacks = week_by_week_df[week_by_week_df["position"] == "QB"]
    runningbacks = week_by_week_df[week_by_week_df["position"] == "RB"]
    wide_receivers = week_by_week_df[week_by_week_df["position"] == "WR"]
    tight_ends = week_by_week_df[week_by_week_df["position"] == "TE"]   
    
    topRunningbacks = runningbacks.groupby("player_name").agg({"fantasy_points_ppr": "mean"}).sort_values("fantasy_points_ppr", ascending=False).head(30)
    topWideReceivers = wide_receivers.groupby("player_name").agg({"fantasy_points_ppr": "mean"}).sort_values("fantasy_points_ppr", ascending=False).head(30)
    topTightEnds = tight_ends.groupby("player_name").agg({"fantasy_points_ppr": "mean"}).sort_values("fantasy_points_ppr", ascending=False).head(20)
    
    topRB_WBW = runningbacks[runningbacks["player_name"].isin(topRunningbacks.index)]
    topWR_WBW = wide_receivers[wide_receivers["player_name"].isin(topWideReceivers.index)]
    topTE_WBW = tight_ends[tight_ends["player_name"].isin(topTightEnds.index)]
    
    
    
    
    
    return matchups_df
    
calculate_player_matchup(schedule2024, week_by_week2024)

32


c:\Users\ieyua\Downloads\Desktop\Github\nflanalytics\functions.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append(team_away_games[['week']].assign(matchup=away_games))
c:\Users\ieyua\Downloads\Desktop\Github\nflanalytics\functions.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append(team_away_games[['week']].assign(matchup=away_games))
c:\Users\ieyua\Downloads\Desktop\Github\nflanalytics\functions.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append(team_away_games[['week']].assign(matchup=away_games))
c:\Users\ieyua\Downloads\Desktop\Github\nflanalytics\functions.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

,team,matchups,qb_opponent_average,rb_opponent_average,wr_opponent_average,te_opponent_average
0,NYJ,"[SF, TEN, NE, DEN, MIN, BUF, PIT, NE, HOU, ARI...",12.941176,19.058824,15.000000,14.470588
1,CHI,"[TEN, HOU, IND, LA, CAR, JAX, WAS, ARI, NE, GB...",15.352941,15.705882,20.823529,15.352941
2,IND,"[HOU, GB, CHI, PIT, JAX, TEN, MIA, HOU, MIN, B...",14.294118,16.411765,14.529412,12.294118
3,BAL,"[KC, LV, DAL, BUF, CIN, WAS, TB, CLE, DEN, CIN...",17.647059,14.294118,14.647059,15.588235
4,LA,"[DET, ARI, SF, CHI, GB, LV, MIN, SEA, MIA, NE,...",11.411765,18.352941,14.941176,13.176471
5,CAR,"[NO, LAC, LV, CIN, CHI, ATL, WAS, DEN, NO, NYG...",19.235294,18.058824,16.117647,19.882353
6,CLE,"[DAL, JAX, NYG, LV, WAS, PHI, CIN, BAL, LAC, N...",17.882353,15.470588,15.000000,19.235294
7,PIT,"[ATL, DEN, LAC, IND, DAL, LV, NYJ, NYG, WAS, B...",20.941176,12.411765,16.647059,18.058824
8,ATL,"[PIT, PHI, KC, NO, TB, CAR, SEA, TB, DAL, NO, ...",17.705882,18.529412,17.058824,19.411765
9,DAL,"[CLE, NO, BAL, NYG, PIT, DET, SF, ATL, PHI, HO...",18.823529,15.411765,20.411765,14.294118


In [42]:
def get_team_schedule (schedule_df,team_name, start_week = 1 , end_week = 18): 
    # filter rows where the team is either the home or away team
    team_home_games = schedule_df[schedule_df['home_team'] == team_name]
    team_away_games = schedule_df[schedule_df['away_team'] == team_name]

    # create lists of matchups
    home_games = [row['away_team'] for _, row in team_home_games.iterrows()]
    away_games = [row['home_team'] for _, row in team_away_games.iterrows()]

    
    # Combine the lists and sort by the 'week' column
    full_schedule = (
        team_home_games[['week']].assign(matchup=home_games)
        .append(team_away_games[['week']].assign(matchup=away_games))
        .sort_values(by='week')
        .reset_index(drop=True)
    )

    # identify bye weeks (weeks where the team is not playing)
    all_weeks = set(range(1, 18))
    played_weeks = set(full_schedule['week'])
    bye_weeks = list(all_weeks - played_weeks)

    # add bye weeks to the schedule
    for bye_week in bye_weeks:
        full_schedule = pd.concat([
            full_schedule,
            pd.DataFrame({"week": [bye_week], "matchup": ["BYE"]})
        ], ignore_index=True)

    full_schedule = full_schedule.sort_values(by='week').reset_index(drop=True)
    schedule_list = full_schedule['matchup'].tolist()

    # account for a specific range of weeks
    sorted_schedule_list = schedule_list[(start_week - 1):(end_week)]

    return sorted_schedule_list
    
    
    
    

team_name = "NYJ" 
nyj_schedule = get_team_schedule(schedule2024, team_name, 11, 18)
print(nyj_schedule)

['IND', 'BYE', 'SEA', 'MIA', 'JAX', 'LA', 'BUF', 'MIA']


C:\Users\ieyua\AppData\Local\Temp\ipykernel_12316\3288331269.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append(team_away_games[['week']].assign(matchup=away_games))


In [13]:
smallWeekByWeek_df = week_by_week2024[['position', 'position_group']]

# getting fantasy defense ranking through week by week data
def fantasy_defense_rankings(week_by_week_df): 
    smallFantasy_df = week_by_week_df[['fantasy_points_ppr', 'position', 'opponent_team', 'player_name', 'week']]
    releventPositions_df = smallFantasy_df[smallFantasy_df['position'].isin(['QB', 'RB', 'WR', 'TE'])]
    
    # using top performers b/c thats usually more indicative of sos in fantasy football 
    topPerformers_df = releventPositions_df.sort_values(by='fantasy_points_ppr', ascending=False).groupby(['opponent_team', 'position']).head(10) # can use .head() if i want to see top performer averages
    
    
    # groupy by opponent team and position in order to find out how many total fantasy points each position has scored against them 
    defensePointsAgainst = topPerformers_df.groupby(['opponent_team', 'position'])['fantasy_points_ppr'].mean().reset_index()
    
    # assign a rank to each team based on the total fantasy points scored against them
    defensePointsAgainst['rank'] = defensePointsAgainst.groupby('position')['fantasy_points_ppr'].rank(ascending=True)
    
    defensePointsAgainst = defensePointsAgainst.sort_values(by=['position', 'rank'])
    
    # pivot the table so that the rankings are in the same row as the team name
    defenseRankings = defensePointsAgainst.pivot(index = "opponent_team", columns = "position", values = "rank")
    defensePPRPoints = defensePointsAgainst.pivot(index = "opponent_team", columns = "position", values = "fantasy_points_ppr")
    
    defenseRankings = defenseRankings.reset_index()
    defensePPRPoints = defensePPRPoints.reset_index()
    
    # merge the rankings and the average ppr points scored against each team
    defenseRankings = defenseRankings.merge(defensePPRPoints, on = "opponent_team", suffixes = ("_rank", "_ppr_avg"))

    return defenseRankings


def merge_defense_schedule (schedule_df, defense_df): 
    home_df = defense_df.add_suffix("_home")
    scheduleWithDefense = schedule_df.merge(home_df, left_on="home_team", right_on="opponent_team_home", how="left", suffixes=("", "_home"))
    scheduleWithDefense = scheduleWithDefense.merge(defense_df, left_on="away_team", right_on="opponent_team", how="left", suffixes=("", "_away"))
    scheduleWithDefense = scheduleWithDefense.drop(columns=["opponent_team_home", "opponent_team_away"], errors="ignore")
    return scheduleWithDefense

test = fantasy_defense_rankings(week_by_week2024)


test

position,opponent_team,QB_rank,RB_rank,TE_rank,WR_rank,QB_ppr_avg,RB_ppr_avg,TE_ppr_avg,WR_ppr_avg
0,ARI,20.0,22.0,10.0,12.0,15.772000,17.160000,8.040000,16.820000
1,ATL,27.0,20.0,25.0,24.0,17.434000,16.830000,11.290000,20.260000
2,BAL,31.0,6.0,28.0,32.0,22.198000,14.520000,12.629999,28.207998
3,BUF,11.0,31.0,9.0,9.0,13.283999,21.439999,7.800000,16.430000
4,CAR,26.0,30.0,26.0,21.0,17.360001,20.580000,11.710000,19.152000
5,CHI,1.0,9.0,16.0,4.0,9.360000,15.150000,9.059999,14.370000
6,CIN,29.0,15.0,31.0,22.0,19.184000,15.990001,13.110001,19.340000
7,CLE,19.0,5.0,3.0,19.0,14.948000,13.630000,6.640000,18.770000
8,DAL,28.0,28.0,15.0,11.0,18.326000,19.340000,9.010000,16.710001
9,DEN,9.0,10.0,20.0,5.0,13.068001,15.270000,9.670000,15.480000


In [ ]:
scheduleWithDefense[['away_team','home_team','QB_rank_home', 'RB_rank_home', 'TE_rank_home',
       'WR_rank_home', 'QB_ppr_avg_home', 'RB_ppr_avg_home', 'TE_ppr_avg_home',
       'WR_ppr_avg_home', 'opponent_team', 'QB_rank', 'RB_rank', 'TE_rank',
       'WR_rank', 'QB_ppr_avg', 'RB_ppr_avg', 'TE_ppr_avg', 'WR_ppr_avg']]

In [ ]:
# creating a function to calculate the win/loss ratio for each team given the seasonal data
def win_loss (schedule_df): 
    # making the dataframe smaller so its easier to handle the data
    smallSchedule2024 = schedule_df[['result', 'away_team', 'away_score', 'home_team', 'home_score', 'week']]   
    previousGames = smallSchedule2024[schedule_df['result'].notnull()]
    
    
    previousGames['home_win'] = np.where(previousGames['result'] > 0, 1, 0)
    previousGames['away_win'] = np.where(previousGames['result'] < 0, 1, 0)
    
    ## not sure if all this is necessary. I think the best way to do it would be to just count the 1s and 0s to figure out the record
    # obtaining the week list
    weekList = previousGames['week'].unique()
    
    #going week by week to calculate the records
    for week in weekList: 
        # getting the data for each week at a time (to tally up the win/loss at the specific time of playing)
        previousWeekData = previousGames[previousGames['week'] <= week]
        
        # calculating the wins and losses for each team
        homeWins = previousWeekData.groupby('home_team')['home_win'].sum()
        awayWins = previousWeekData.groupby('away_team')['away_win'].sum()
        homeLosses = previousWeekData.groupby('home_team')['home_win'].count() - previousWeekData.groupby('home_team')['home_win'].sum()
        awayLosses = previousWeekData.groupby('away_team')['away_win'].count() - previousWeekData.groupby('away_team')['away_win'].sum()
        
        teamWins = homeWins.add(awayWins, fill_value=0)
        teamLosses = homeLosses.add(awayLosses, fill_value=0)
        
        # creates a mask so we get the right week in the origianl df
        mask = schedule_df['week'] == week

        # adding the win/loss record to the schedule_df
        schedule_df.loc[mask, 'home_team_win_record'] = schedule_df['home_team'].map(teamWins)
        schedule_df.loc[mask, 'home_team_loss_record'] = schedule_df['home_team'].map(teamLosses)
        schedule_df.loc[mask, 'away_team_win_record'] = schedule_df['away_team'].map(teamWins)
        schedule_df.loc[mask, 'away_team_loss_record'] = schedule_df['away_team'].map(teamLosses)

    return schedule_df


scheduleWithRecords = win_loss(schedule2024) 
previousGames = scheduleWithRecords[scheduleWithRecords['result'].notnull()]
previousGames
